# Financial Tweet NER and Stock Symbol Identification

This notebook processes the raw tweet CSV files to:
1. Apply BERT-based NER to identify entity types in tweets
2. Extract potential stock symbols and match them with actual stock symbols
3. Prepare the data for sentiment labeling

These preprocessing steps will help identify tweets that are discussing specific stocks.

In [1]:
import os
import pandas as pd
import numpy as np
import time
import re
from glob import glob
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 1000)

## 1. Install Required Packages

First, we need to install the necessary packages for NER and stock symbol lookup.

In [2]:
!pip install transformers yfinance requests

  Using cached transformers-4.50.2-py3-none-any.whl.metadata (39 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached peewee-3.17.9.tar.gz (3.0 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached transformers-4.50.2-py3-none-any.whl (10.2 MB)
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl (418 kB)
  Created wheel for peewee: filename=peewee-3.17.9-cp311-cp311-macosx_11_0_arm64.whl size=264594 sha256=31bf562b81d4ca1a53fb13f326be0da982bc1dbf9dca111a0d17b72db9e4f0d9
  Stored in directory: /Users/jethrotsoi/Library/Caches/pip/wheels/f4/14/e4/50c88c865833085aeb91e2bd40e3a683ff434806386b8ee7bc
Successfully built peewee


In [3]:
# Import the newly installed packages
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import yfinance as yf
# Create output directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## 2. Load Raw Tweet Files

Load all the raw CSV files from the data/tweets directory.

In [4]:
def load_tweet_files(data_dir='../data/tweets/'):
    """Load all row_*.csv files from the specified directory"""
    # Check if directory exists
    if not os.path.exists(data_dir):
        print(f"Warning: Directory {data_dir} does not exist. Creating it.")
        os.makedirs(data_dir, exist_ok=True)
        return pd.DataFrame()
        
    all_files = glob(os.path.join(data_dir, 'row_*.csv'))
    
    if not all_files:
        print(f"Warning: No row_*.csv files found in {data_dir}")
        return pd.DataFrame()
    
    dataframes = []
    for file in tqdm(all_files, desc='Loading files'):
        try:
            df = pd.read_csv(file)
            # Add source file name as a column
            df['source_file'] = os.path.basename(file)
            dataframes.append(df)
        except Exception as e:
            print(f"Error loading {file}: {str(e)}")
    
    if dataframes:
        return pd.concat(dataframes, ignore_index=True)
    else:
        return pd.DataFrame()

# Load all tweet data
df = load_tweet_files()
print(f"Total tweets loaded: {len(df)}")
df.head()

Loading files: 100%|██████████| 61/61 [00:00<00:00, 135.27it/s]

Total tweets loaded: 46656


,id,created_at,full_text,media,screen_name,name,profile_image_url,in_reply_to,retweeted_status,quoted_status,favorite_count,retweet_count,bookmark_count,quote_count,reply_count,views_count,favorited,retweeted,bookmarked,url,source_file
0,1476040410539560960,2021-12-29 12:00:32 +08:00,Savings is overated\nIncreasing income is unde...,[],MashraniVivek,"Vivek Mashrani, CFA",https://pbs.twimg.com/profile_images/157292460...,NaN,NaN,NaN,1747,145,39,14,40,NaN,False,False,False,https://twitter.com/MashraniVivek/status/14760...,row_28_mashranivivek.csv
1,1756238277839884344,2024-02-10 16:46:49 +08:00,Ever felt like you're riding a rollercoaster w...,"[{""type"":""photo"",""url"":""https://t.co/VRqP4DxfZ...",MashraniVivek,"Vivek Mashrani, CFA",https://pbs.twimg.com/profile_images/157292460...,NaN,NaN,NaN,31,5,14,0,5,6404.0,False,False,False,https://twitter.com/MashraniVivek/status/17562...,row_28_mashranivivek.csv
2,1756238318755365117,2024-02-10 16:46:59 +08:00,8. Strategic Allocation Across Sectors:\n\nAll...,[],MashraniVivek,"Vivek Mashrani, CFA",https://pbs.twimg.com/profile_images/157292460...,1.756238e+18,NaN,NaN,3,1,0,0,1,506.0,False,False,False,https://twitter.com/MashraniVivek/status/17562...,row_28_mashranivivek.csv
3,1756238321980768720,2024-02-10 16:46:59 +08:00,9. Consider Individual Risk Tolerance:\n\nTail...,[],MashraniVivek,"Vivek Mashrani, CFA",https://pbs.twimg.com/profile_images/157292460...,1.756238e+18,NaN,NaN,2,1,0,0,1,2685.0,False,False,False,https://twitter.com/MashraniVivek/status/17562...,row_28_mashranivivek.csv
4,1756238325155803182,2024-02-10 16:47:00 +08:00,10. Implement a Systematic Approach to Manage ...,[],MashraniVivek,"Vivek Mashrani, CFA",https://pbs.twimg.com/profile_images/157292460...,1.756238e+18,NaN,NaN,4,2,0,0,0,2655.0,False,False,False,https://twitter.com/MashraniVivek/status/17562...,row_28_mashranivivek.csv


## 3. Text Cleaning for NER

Clean the tweet text before applying NER, but preserve mentions of potential stock symbols.

In [5]:
def clean_tweet_for_ner(text):
    """Clean tweet text for NER while preserving potential stock symbols"""
    if pd.isna(text):
        return ""
    
    # Convert to string if not already
    text = str(text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Replace extra whitespace
    text = ' '.join(text.split())
    
    return text.strip()

# Clean the tweets
df['cleaned_text'] = df['full_text'].apply(clean_tweet_for_ner)

# Remove empty tweets
df = df[df['cleaned_text'].str.len() > 0].reset_index(drop=True)
print(f"Tweets after cleaning: {len(df)}")
df[['cleaned_text']].head()

Tweets after cleaning: 45875


,cleaned_text
0,Savings is overated\nIncreasing income is unde...
1,Ever felt like you're riding a rollercoaster w...
2,8. Strategic Allocation Across Sectors:\n\nAll...
3,9. Consider Individual Risk Tolerance:\n\nTail...
4,10. Implement a Systematic Approach to Manage ...


## 4. Apply BERT-based NER

Use BERT-based Named Entity Recognition to identify entities in the tweets.

In [6]:
# Load NER pipeline
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

ImportError: 
AutoModelForTokenClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:
# Apply NER to all tweets (this may take some time)
print("Applying NER to all tweets. This may take a while...")

# Check for checkpoint file
checkpoint_file = '../data/ner_checkpoint.csv'
if os.path.exists(checkpoint_file):
    try:
        checkpoint_df = pd.read_csv(checkpoint_file)
        if 'entity_types' in checkpoint_df.columns and 'entity_values' in checkpoint_df.columns:
            print(f"Loading NER results from checkpoint file ({len(checkpoint_df)} rows)")
            df = checkpoint_df
            ner_complete = True
        else:
            ner_complete = False
    except Exception as e:
        print(f"Error loading checkpoint: {str(e)}")
        ner_complete = False
else:
    ner_complete = False

if not ner_complete:
    # We'll process in smaller batches to prevent memory issues
    batch_size = 100
    num_batches = (len(df) + batch_size - 1) // batch_size
    
    all_entity_types = []
    all_entity_values = []
    
    for i in tqdm(range(num_batches), desc="Processing batches"):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(df))
        
        batch_texts = df['cleaned_text'].iloc[start_idx:end_idx].tolist()
        batch_results = []
        
        for text in batch_texts:
            batch_results.append(apply_ner(text, ner_pipeline))
        
        all_entity_types.extend([r[0] for r in batch_results])
        all_entity_values.extend([r[1] for r in batch_results])
        
        # Save checkpoint every 5 batches
        if (i+1) % 5 == 0 or i == num_batches - 1:
            temp_df = df.copy()
            temp_types = all_entity_types + [[] for _ in range(len(df) - len(all_entity_types))]
            temp_values = all_entity_values + [[] for _ in range(len(df) - len(all_entity_values))]
            temp_df['entity_types'] = temp_types
            temp_df['entity_values'] = temp_values
            temp_df.to_csv(checkpoint_file, index=False)
            print(f"Saved checkpoint after batch {i+1}/{num_batches}")
    
    # Add results to dataframe
    df['entity_types'] = all_entity_types
    df['entity_values'] = all_entity_values

print("NER processing complete!")
df[['cleaned_text', 'entity_types', 'entity_values']].head()

In [ ]:
# Apply NER to all tweets (this may take some time)
print("Applying NER to all tweets. This may take a while...")

# We'll process in smaller batches to prevent memory issues
batch_size = 100
num_batches = (len(df) + batch_size - 1) // batch_size

all_entity_types = []
all_entity_values = []

for i in tqdm(range(num_batches), desc="Processing batches"):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(df))
    
    batch_texts = df['cleaned_text'].iloc[start_idx:end_idx].tolist()
    batch_results = []
    
    for text in batch_texts:
        batch_results.append(apply_ner(text, ner_pipeline))
    
    all_entity_types.extend([r[0] for r in batch_results])
    all_entity_values.extend([r[1] for r in batch_results])

# Add results to dataframe
df['entity_types'] = all_entity_types
df['entity_values'] = all_entity_values

print("NER processing complete!")
df[['cleaned_text', 'entity_types', 'entity_values']].head()

## 5. Extract and Verify Stock Symbols

Extract potential stock symbols from tweets and verify them against actual stock symbols.

In [ ]:
def verify_stock_symbol(symbol, max_retries=3):
    """Verify if a given string is a valid stock symbol using yfinance with retry logic"""
    for attempt in range(max_retries):
        try:
            ticker = yf.Ticker(symbol)
            # Try to get some basic info to verify it's valid
            info = ticker.info
            
            # Check if we got valid info (if the ticker doesn't exist, this will be empty or have minimal data)
            if 'symbol' in info and info['symbol'] == symbol:
                return True
            return False
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Retry {attempt+1} for {symbol}: {str(e)}")
                time.sleep(1)  # Wait before retrying
            else:
                print(f"Failed to verify {symbol} after {max_retries} attempts: {str(e)}")
                return False

def get_verified_symbols(symbols_list):
    """From a list of potential symbols, return the valid ones"""
    if not symbols_list:
        return None
    
    verified = []
    for symbol in symbols_list:
        if verify_stock_symbol(symbol):
            verified.append(symbol)
    
    return verified if verified else None

# Test on a small sample first
sample_symbols = df[df['potential_symbols'].apply(lambda x: len(x) > 0)]['potential_symbols'].head(5)
for symbols in sample_symbols:
    verified = get_verified_symbols(symbols)
    print(f"Potential: {symbols}, Verified: {verified}")

In [ ]:
# Apply verification to all potential symbols
print("Verifying stock symbols. This may take a while...")

# Check for checkpoint file
symbols_checkpoint_file = '../data/symbols_checkpoint.csv'
if os.path.exists(symbols_checkpoint_file):
    try:
        symbols_df = pd.read_csv(symbols_checkpoint_file)
        if 'verified_stock_symbols' in symbols_df.columns:
            print(f"Loading verified symbols from checkpoint file ({len(symbols_df)} rows)")
            df = symbols_df
            symbols_complete = True
        else:
            symbols_complete = False
    except Exception as e:
        print(f"Error loading symbols checkpoint: {str(e)}")
        symbols_complete = False
else:
    symbols_complete = False

if not symbols_complete:
    # Create a cache to avoid repeated API calls for the same symbol
    symbol_cache = {}
    
    def get_verified_symbols_with_cache(symbols_list):
        """Use cache to speed up symbol verification"""
        if not symbols_list:
            return None
        
        verified = []
        for symbol in symbols_list:
            if symbol in symbol_cache:
                if symbol_cache[symbol]:
                    verified.append(symbol)
            else:
                is_valid = verify_stock_symbol(symbol)
                symbol_cache[symbol] = is_valid
                if is_valid:
                    verified.append(symbol)
        
        return verified if verified else None
    
    # Process in batches and save checkpoints
    batch_size = 100
    rows_with_symbols = df[df['potential_symbols'].apply(lambda x: len(x) > 0)]
    
    if len(rows_with_symbols) > 0:
        print(f"Processing {len(rows_with_symbols)} tweets with potential symbols")
        
        for i, idx in enumerate(tqdm(rows_with_symbols.index, desc="Verifying symbols")):
            symbols = df.loc[idx, 'potential_symbols']
            verified = get_verified_symbols_with_cache(symbols)
            df.at[idx, 'verified_stock_symbols'] = verified
            
            # Save checkpoint periodically
            if (i+1) % batch_size == 0 or i == len(rows_with_symbols) - 1:
                df.to_csv(symbols_checkpoint_file, index=False)
                print(f"Saved checkpoint after processing {i+1}/{len(rows_with_symbols)} rows")
    else:
        print("No tweets with potential stock symbols found")

df[['cleaned_text', 'potential_symbols', 'verified_stock_symbols']].head(10)

In [ ]:
# Apply verification to all potential symbols
print("Verifying stock symbols. This may take a while...")

# Create a cache to avoid repeated API calls for the same symbol
symbol_cache = {}

def get_verified_symbols_with_cache(symbols_list):
    """Use cache to speed up symbol verification"""
    if not symbols_list:
        return None
    
    verified = []
    for symbol in symbols_list:
        if symbol in symbol_cache:
            if symbol_cache[symbol]:
                verified.append(symbol)
        else:
            is_valid = verify_stock_symbol(symbol)
            symbol_cache[symbol] = is_valid
            if is_valid:
                verified.append(symbol)
    
    return verified if verified else None

# Apply to all rows with potential symbols
tqdm.pandas(desc="Verifying symbols")
df['verified_stock_symbols'] = df['potential_symbols'].progress_apply(get_verified_symbols_with_cache)
df[['cleaned_text', 'potential_symbols', 'verified_stock_symbols']].head(10)

## 6. Summary of Results

In [ ]:
# Save all processed tweets
os.makedirs('../data', exist_ok=True)  # Ensure directory exists
df.to_csv('../data/tweets_with_ner_and_stocks.csv', index=False)
print(f"Saved all processed tweets to '../data/tweets_with_ner_and_stocks.csv'")

# Create a filtered dataset with only tweets that mention verified stock symbols
if 'verified_stock_symbols' in df.columns:
    stock_tweets_df = df[df['verified_stock_symbols'].notnull()].copy()
    stock_tweets_df.to_csv('../data/tweets_with_verified_stocks.csv', index=False)
    print(f"Saved {len(stock_tweets_df)} tweets with verified stock symbols to '../data/tweets_with_verified_stocks.csv'")
else:
    print("Warning: No 'verified_stock_symbols' column found in the dataframe. Check the previous steps.")

## 7. Save Processed Data for Labeling

Save the processed data with NER results and verified stock symbols.

In [ ]:
# Save all processed tweets
df.to_csv('../data/tweets_with_ner_and_stocks.csv', index=False)
print(f"Saved all processed tweets to '../data/tweets_with_ner_and_stocks.csv'")

# Create a filtered dataset with only tweets that mention verified stock symbols
stock_tweets_df = df[df['verified_stock_symbols'].notnull()].copy()
stock_tweets_df.to_csv('../data/tweets_with_verified_stocks.csv', index=False)
print(f"Saved {len(stock_tweets_df)} tweets with verified stock symbols to '../data/tweets_with_verified_stocks.csv'")

## 8. Next Steps

After this notebook, you can proceed to the sentiment labeling process using:
1. The `tweets_with_verified_stocks.csv` file for tweets that mention specific stocks
2. The `tweets_with_ner_and_stocks.csv` file for all tweets with NER information

The sentiment labeling should now focus on the tweets with verified stock symbols.